In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings

In [2]:
from sklearn.linear_model import Lasso

In [3]:
import dask.dataframe as dd

На этой неделе вам предстоит попробовать добавить в вашу регрессионную модель дополнительные признаки.

Во-первых, для прогнозирования можно использовать информацию, содержащуюся в сырых данных:

* средняя длительность поездок
* среднее количество пассажиров
* среднее расстояние по счётчику
* доли географических зон, в которые совершаются поездки
* доли поездок, совершаемых по тарифам каждого из типов
* доли способов оплаты поездок
* средняя стоимость поездок
* доли провайдеров данных
* Все эти признаки можно использовать только с задержкой, то есть, при прогнозировании $\hat{y}_{T+i|T}$

эти признаки должны быть рассчитаны по данным не позднее момента времени $T$. Каждый из этих признаков можно использовать по-разному: как сырые значения за последние несколько часов, так и средние за последний день, неделю, месяц и т. д.

Во-вторых, чтобы улучшить качество прогнозов в аномальные периоды, вы можете найти информацию о потенциально влияющих на количество поездок событиях, таких, как государственные праздники. Проанализируйте, как именно поведение пассажиров меняется во время этих событий, и создайте признаки, отражающие эти изменения. Как показывает наш опыт, правильный учёт праздничных дней часто позволяет существенно уменьшить среднюю ошибку прогноза.

В-третьих, можно использовать признаки, связанные с географией. Например, скорее всего, суммарное количество поездок, совершаемых из географической зоны, пропорционально площади этой зоны. Для зон, прилегающих к аэропорту, может быть характерен специфический паттерн дневной сезонности, связанный с тем, что спрос на такси будет повышаться в те часы, когда общественный транспорт перестаёт работать. В деловом центре максимальное количество поездок будет приходиться на начало и окончание рабочего дня, на Бродвее — на время начала и окончания спектаклей. Все эти идеи не обязательно верны, мы приводим их здесь только для того, чтобы продемонстрировать принцип рассуждений. Ещё один пример географического признака: можно попробовать добавить идентификатор боро, который можно найти в файле https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv. Кроме того, нам кажется перспективным использование в качестве фактора количества поездок, совершённых за прошлый час/день и т. д. из соседних географических зон, или количества поездок, совершённых за прошлый час/день в текущую географическую зону.

Много примеров других признаков, которые можно использовать при регрессионном прогнозировании, можно найти в лекции Вадима Стрижова.

Чтобы сдать задание, выполните следующую последовательность действий.

1. Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте $Q_{may}$ — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.

Подгрузим данные, которые мы собрали на прошлой неделе

In [4]:
april_X = pd.read_csv('april_X.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])
may_X = pd.read_csv('may_X.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])
june_X = pd.read_csv('june_X.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])

train_ys = pd.read_csv('train_ys.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])
test_ys = pd.read_csv('test_ys.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])
valid_ys = pd.read_csv('valid_ys.csv', index_col=['tpep_pickup_datetime', 'Unnamed: 1'])

Тогда я использовал обычную лассо модель без тюнинга параметров, так что теперь это будет нашим бейзлайном и будем улучшать результат заменой на более продвинутые алгоритмы и улучшением признаков

In [5]:
simple_models = [Lasso().fit(april_X, train_ys[f'T+{i}'].values) for i in range(1, 7)]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1057255825.5962633, tolerance: 5153865.000252157
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2633419553.7985444, tolerance: 5153742.5031987345
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2874149380.6548967, tolerance: 5153328.431354502
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

### Признаки

* T - Количество поездок в текущий момент времени
* T-1 - Количество поездок час назад
* T-2 - Количество поездок 2 часа назад
* T-3 - Количество поездок 3 часа назад
* T-4 - Количество поездок 4 часа назад
* T-5 - Количество поездок 5 часов назад
* T-6 - Количество поездок 6 часов назад
* T-24 - Количество поездок в тот же час, но на день раньше
* T-48 - Количество поездок в тот же час, но на 2 дня раньше
* day - календарный день
* day_str - календарный день в строковом виде
* dayofweek - календарный день недели
* dayofweek_str - календарный день недели в строковом виде
* hour - час
* hour_str - час в строковом виде
* month - календарный месяц
* month_str - календарный месяц в строковом виде
* year - календарный год
* year_str - календарный год в строковом виде
* region - номер региона в строком виде
* halfday_sum - количество поездок за прошедшие полдня
* day_sum - количество поездок за прошедший день
* week_sum - количество поездок за прошедшую неделю
* month_sum - количество поездок за прошедший месяц

Сделаем прогноз и посчитаем ошибку, чтобы в дальнейшем на нее ориентироваться

In [6]:
predicts = np.array([model.predict(may_X) for model in simple_models]).T
predicts[predicts < 0] = 0

In [7]:
np.mean(np.abs(test_ys.values - np.array(predicts)))

41.20619307898187

2. Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.

Данных у нас очень много и чтобы упростить загрузка и обработку, мы используем Dask - аналог Pandas, только легко параллелится и загружает данные не полностью, что позволяет "обрабатывать" все сразу

In [8]:
raw_data = dd.read_csv('yellow_tripdata_*.csv', parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [9]:
raw_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-05-05 23:37:40,2015-05-05 23:45:41,1,2.00,-74.001678,40.739311,1,N,-73.978294,40.752110,2,8.5,0.5,0.5,0.00,0.0,0.3,9.80
1,2,2015-05-05 23:37:40,2015-05-05 23:40:36,1,0.54,-73.930840,40.744789,1,N,-73.937515,40.749359,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
2,2,2015-05-05 23:37:40,2015-05-05 23:44:03,3,2.10,-74.001411,40.731087,1,N,-73.981674,40.758282,2,8.0,0.5,0.5,0.00,0.0,0.3,9.30
3,2,2015-05-05 23:37:40,2015-05-06 00:14:01,6,10.93,-73.970673,40.758560,1,N,-73.933762,40.670544,1,36.0,0.5,0.5,9.32,0.0,0.3,46.62
4,2,2015-05-05 23:37:40,2015-05-05 23:46:03,5,0.93,-73.986732,40.755878,1,N,-73.990959,40.749981,1,7.0,0.5,0.5,2.49,0.0,0.3,10.79


* удалим поездки с нулевой или отрицательной длительностью

In [10]:
raw_data = raw_data[(raw_data['tpep_dropoff_datetime'] - raw_data['tpep_pickup_datetime']).dt.seconds > 0]

* удалим поездки с нулевым или отрицательным количеством пассажиров

In [11]:
raw_data = raw_data[raw_data['passenger_count'] > 0]

* удалим поездки с нулевым или отрицательным расстоянием

In [12]:
raw_data = raw_data[raw_data['trip_distance'] > 0]

* удалим поездки с непопадающим началом в прямоугольник Нью-Йорка

In [13]:
raw_data = raw_data[
    (-74.25559 <= raw_data['pickup_longitude']) &
    (raw_data['pickup_longitude'] <= -73.70001) &
    (40.49612 <= raw_data['pickup_latitude']) &
    (raw_data['pickup_latitude'] <= 40.91553)
]

Загрузим границы регионов и оставим только нужные

In [14]:
regions = pd.read_csv('regions.csv', sep=';')
regions = regions[regions['region'].isin(april_X.index.levels[1])]
regions.head()

,region,west,east,south,north
1074,1075,-74.022246,-74.011135,40.697437,40.705825
1075,1076,-74.022246,-74.011135,40.705825,40.714213
1076,1077,-74.022246,-74.011135,40.714213,40.722601
1124,1125,-74.011135,-74.000023,40.697437,40.705825
1125,1126,-74.011135,-74.000023,40.705825,40.714213


Создадим колонку, заранее заполнив его NaN, это поможет, чтобы убрать поездки, которые не входят в неинтересные регионы

In [15]:
raw_data['region'] = np.nan

Присвоим номер региона соответствующим поездкам

In [16]:
for _, row in regions.iterrows():
    raw_data['region'] = raw_data['region'].mask(
        (row['west'] <= raw_data['pickup_longitude']) &
        (row['east'] >= raw_data['pickup_longitude']) &
        (row['south'] <= raw_data['pickup_latitude']) &
        (row['north'] <= raw_data['pickup_latitude']),
        row['region'])
#     raw_data.loc[
#         (row['west'] <= raw_data['pickup_longitude']) &
#         (row['east'] >= raw_data['pickup_longitude']) &
#         (row['south'] <= raw_data['pickup_latitude']) &
#         (row['north'] <= raw_data['pickup_latitude']),
#         'region'
#     ] = row['region']

In [21]:
raw_data = raw_data.dropna()

In [ ]:
raw_data.groupby('region').passenger_count.sum().compute()

In [ ]:
raw_data['region'].drop_duplicates().head(2)

In [ ]:
raw_data['region'].unique().compute()

Оставим только интересные нам регионы

In [ ]:
raw_data = raw_data[april_X.index.levels[1].astype(str)]

3. Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

$$
Q_{may}=\frac{1}{R*739*6}\sum_{r=1}^{R}{\sum_{T=2016.04.30\ 23:00}^{2016.05.31\ 17:00}{\sum_{i=1}^{6}{|\hat{y}^r_{T|T+i} - y^r_{T+i}|}}}
$$

Убедитесь, что среднее качество прогнозов увеличилось.

In [8]:
np.mean(np.abs(test_ys.values - np.array(predicts)))

41.20619307898187

4. Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: *geoID, histEndDay, histEndHour, step, y*

In [9]:
predicts = np.array([model.predict(june_X) for model in simple_models]).T
predicts[predicts < 0] = 0

In [16]:
valid_predict = pd.DataFrame(np.array([model.predict(june_X) for model in simple_models]).T, index=valid_ys.index)
valid_predict = valid_predict.stack().reset_index()
valid_predict['tpep_pickup_datetime'] = pd.to_datetime(valid_predict['tpep_pickup_datetime'])

def rewrite_month(date):
    return '0' + str(date.month) if date.month < 10 else str(date.month)

def rewrite_day(date):
    return '0' + str(date.day) if date.day < 10 else str(date.day)

reg_date = valid_predict['level_1'].astype(str) + \
        '_' + valid_predict['tpep_pickup_datetime'].dt.year.astype(str) + \
        '-'+ valid_predict['tpep_pickup_datetime'].apply(rewrite_month) + \
        '-' + valid_predict['tpep_pickup_datetime'].apply(rewrite_day) + \
        '_' + valid_predict['tpep_pickup_datetime'].dt.hour.astype(str) + \
        '_' + (valid_predict['level_2'] + 1).astype(str)

valid_predict['id'] = reg_date
valid_predict = valid_predict[['id', 0]]
valid_predict.columns = ['id', 'y']

valid_predict.to_csv('to_kaggle_week6.csv', index=False)

5. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

Score: 39.89764

6. Загрузите ноутбук в форму

Done :3